In [1]:
import json
import requests
import threading
import re

URL = 'http://35.188.227.39:8080/enhancer/chain/ReceiptChain'
LABEL = 'http://fise.iks-project.eu/ontology/entity-reference'
SELECTED_LABEL = 'http://bizlem.io/PurchaseOrderProcessing#'

In [2]:
def middle(value, left, right):
    return value >= left and value <= right

def get_x(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['x']

def get_y(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['y']

def get_box(obj):
    left_x = max(get_x(obj, 0), get_x(obj, 3))
    right_x = min(get_x(obj, 1), get_x(obj, 2))
    top_y = max(get_y(obj, 0), get_y(obj, 1))
    bot_y = min(get_y(obj, 2), get_y(obj, 3))
    
    return (left_x, top_y, right_x, bot_y)

def get_x_range(arr_obj, boxes):
    left_x, _, _, _ = boxes[arr_obj[0]]
    _, _, right_x, _ = boxes[arr_obj[-1]]
    return (left_x, right_x)

def to_float(s):
    s_format = ''.join(re.split('[^\d\.]', s))
    return float(s_format)

In [43]:
folder = 'c10'
path = 'p4materials/' + folder + '.json'
with open(path) as f:
    data = json.load(f)
    data_sum = data[0]
    data = data[1:]

    max_x = 0
    max_y = 0
    
# calculate boxes
boxes = []
for obj in data:
    boxes.append(get_box(obj))

# find max_x and max_y
for i, obj in enumerate(data):
    _, _, right_x, bot_y = boxes[i]
    max_x = max(max_x, right_x)
    max_y = max(max_y, bot_y)
    
print(max_x, max_y, len(data))
height = max_y + 1
width = max_x + 1
coor = [[ None for x in range(width) ] for y in range(height)] 

# map to coor
for index, obj in enumerate(data):
    left_x, top_y, right_x, bot_y = boxes[index]

    for y in range(top_y, bot_y + 1):
        for x in range(left_x, right_x + 1):
            if coor[y][x] is not None:
                continue
            coor[y][x] = index

2325 3409 370


In [48]:
slist = {}
visited = []

MARGIN_X = 5
MARGIN_Y = 6

print('processing matrix...')
for x in range(width):
    for y in range(height - 1, -1, -1):
        if coor[y][x] is None or coor[y][x] in visited:
            continue
            
        index_obj = coor[y][x]
        left_x, top_y, _, bot_y = boxes[index_obj]
        obj = data[index_obj]
        visited.append(index_obj)
        
        # test range y within MARGIN_Y
        row = y
        MARGIN_Y = int((bot_y - top_y) * 0.7)
        for i in range(y - MARGIN_Y, y + MARGIN_Y + 1):
            if i in slist.keys():
                row = i
                break
        
        if row not in slist.keys():
            slist[row] = []
            
        if len(slist[row]) > 0:
            prev_words = slist[row][-1]
            prev_word = prev_words[-1]
            pw_left_x, pw_top_y, pw_right_x, pw_bot_y = boxes[prev_word]
            
            MARGIN_X = int((pw_bot_y - pw_top_y) * 0.7)
            
            if pw_right_x + MARGIN_X >= left_x:
                prev_words.append(index_obj)
                continue
            
        slist[row].append([ index_obj ])
print('done processing matrix')

processing matrix...
done processing matrix


In [49]:
lines = list(slist.keys())
lines.sort()

for line in lines:
    words_list = slist[line]
    temp = []
    for words in words_list:
        sentence = [ data[index]['description'] for index in words ]
        temp.append(' '.join(sentence))
    print(line, temp)

234 ['Shree /', 'Karthik Engineering', 'TAX INVOICE']
315 ['263 264 , G I DC , Ichchapore - Bhatpore ,', 'Phone No . : + 919825800890']
380 ['Near GAIL , SURAT - 394 510 .', 'Email : shreekarthikengg @ gmail . com']
448 ['GSTIN : 24ABIFS0267Q1ZO']
520 ['PAN NO : ABIFS0267Q', 'Page 1 of 1']
623 ['TO :', 'CONSIGNEE ( Shipped To – Address of Delivery )']
683 ['ESSAR STEEL INDIA LIMITED', 'Order No .', 'ESTL - DTA / SCC / 4400116403']
757 ['27TH KM , SURAT - HAZIRA ROAD', '102100', 'Order Date', '26 / 09 / 2017']
823 ['HAZIRA , SURAT', 'Invoice No .', '1819960']
892 ['SUART GUJARAT 394270', 'Invoice Date 25 / 12 / 2018']
954 ['GSTIN NO : 24AAACE1741P1ZN', 'PAN NO : AAACE1741P', 'Vendor Code 102100']
1066 ['SAC Code', 'MAINTENANCE AND', 'REPAIR', 'SERVICES OF PLACE OF SUPPLY STATE CODE', 'ORIGINAL FOR']
1099 ['098717', 'COMMERCIAL AND INDUSTRIAL MACHINERY |', 'HAZIRA . SURAT', '| 24']
1122 ['GUJARATI', 'RECIPIENT']
1233 ['Sr .', 'Rate Per']
1288 ['No .', 'Description', 'Unit', 'Unit', 'Qty'

In [50]:
ssum = {}
sres = {}

for line in lines:
    words_list = slist[line]
    phrases = []
    for words in words_list:
        phrase = [ data[index]['description'] for index in words ]
        phrases.append(' '.join(phrase))
    ssum[line] = phrases
    sres[line] = phrases.copy()
    

In [51]:
for line, words in ssum.items():
    print(line, words)

234 ['Shree /', 'Karthik Engineering', 'TAX INVOICE']
315 ['263 264 , G I DC , Ichchapore - Bhatpore ,', 'Phone No . : + 919825800890']
380 ['Near GAIL , SURAT - 394 510 .', 'Email : shreekarthikengg @ gmail . com']
448 ['GSTIN : 24ABIFS0267Q1ZO']
520 ['PAN NO : ABIFS0267Q', 'Page 1 of 1']
623 ['TO :', 'CONSIGNEE ( Shipped To – Address of Delivery )']
683 ['ESSAR STEEL INDIA LIMITED', 'Order No .', 'ESTL - DTA / SCC / 4400116403']
757 ['27TH KM , SURAT - HAZIRA ROAD', '102100', 'Order Date', '26 / 09 / 2017']
823 ['HAZIRA , SURAT', 'Invoice No .', '1819960']
892 ['SUART GUJARAT 394270', 'Invoice Date 25 / 12 / 2018']
954 ['GSTIN NO : 24AAACE1741P1ZN', 'PAN NO : AAACE1741P', 'Vendor Code 102100']
1066 ['SAC Code', 'MAINTENANCE AND', 'REPAIR', 'SERVICES OF PLACE OF SUPPLY STATE CODE', 'ORIGINAL FOR']
1099 ['098717', 'COMMERCIAL AND INDUSTRIAL MACHINERY |', 'HAZIRA . SURAT', '| 24']
1122 ['GUJARATI', 'RECIPIENT']
1233 ['Sr .', 'Rate Per']
1288 ['No .', 'Description', 'Unit', 'Unit', 'Qty'

In [52]:
def request_word(word):
    res = []
    try:
        r = requests.post(URL, data=word.encode('utf-8'), headers={'Content-Type': 'application/pdf'})
        r = r.json()
        
        for obj in r:
            if LABEL in obj:
                res.append(obj[LABEL][0]['@id'])
        return res
    except Exception as e:
        print(word)
        print(e)
        return res
    
def request_label(ssum, line, index, sres):
    res = request_word(ssum[line][index])
    sres[line][index] = res
    
def request_line(ssum, line, sres):
    threads = [ threading.Thread(target=request_label, args=(ssum, line, index, sres)) for index in range(len(ssum[line])) ]
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()
        
def request_all(ssum, sres):
    threads = [ threading.Thread(target=request_line, args=(ssum, line, sres)) for line in ssum.keys() ]
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()
    
print('requesting...')
request_all(ssum, sres)
print('finish requesting...')

requesting...
finish requesting...


In [40]:
r = request_word('TAX INVOICE')
print(r)

['http://bizlem.io/PurchaseOrderProcessing#Tax_Invoice']


In [53]:
for line, words in sres.items():
    print(line, words)

234 [[], [], ['http://bizlem.io/PurchaseOrderProcessing#Tax_Invoice']]
315 [[], []]
380 [[], []]
448 [['http://bizlem.io/PurchaseOrderProcessing#GSTIN']]
520 [['http://bizlem.io/PurchaseOrderProcessing#PAN_NO'], []]
623 [[], ['http://bizlem.io/PurchaseOrderProcessing#Ship_To']]
683 [[], [], []]
757 [['http://bizlem.io/PurchaseOrderProcessing#Hazira', 'http://bizlem.io/Quantity#km'], [], [], []]
823 [['http://bizlem.io/PurchaseOrderProcessing#Hazira'], ['http://bizlem.io/PurchaseOrderProcessing#INVOICE_NO'], []]
892 [[], ['http://bizlem.io/PurchaseOrderProcessing#Invoice_Date']]
954 [['http://bizlem.io/PurchaseOrderProcessing#GSTIN'], ['http://bizlem.io/PurchaseOrderProcessing#PAN_NO'], []]
1066 [['http://bizlem.io/PurchaseOrderProcessing#HSN%2FSAC'], [], [], [], []]
1099 [[], [], ['http://bizlem.io/PurchaseOrderProcessing#Hazira'], []]
1122 [[], []]
1233 [[], ['http://bizlem.io/PurchaseOrderProcessing#RATE']]
1288 [[], ['http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES'

In [10]:
def concat(ssum, line, index):                            
    paragraph = ''                                         
    for l, phrases in ssum.items():                         
        if l < line:                                       
            continue
        if l == line:
            paragraph += ' '.join(phrases[index:])
        else:
            sentence = ' '.join(phrases)
            paragraph += '\n' + sentence
    return paragraph

def strip(s):
    return ''.join(re.split('[^a-zA-Z0-9]', s.lower()))

In [54]:
date_regex = '\d{1,2}\s?[\.\-\/\:]\s?\d{1,2}\s?[\.\-\/\:]\s?\d{2,4}|\d{1,2}\s?\-?\s?[A-Za-z]{3,}\s?\-?\s?(\d{2}|\d{4})'
number_regex = '(\,? ?\d+ ?)+(\. ?\d+)?(?!\d{0,} ?%)'
percentage_regex = '\d+\s?%|\d+\s?\.\s?%'

# processing text
patterns = {                                               
    'pan': {                                               
        'regex': '[A-Z]{5}[0-9]{4}[A-Z]{1}',               
        'label': 'PAN_NO',
        'main_key': 'PAN_NO',
    },
    'tan': {                                               
        'regex': '[A-Z]{4}[0-9]{5}[A-Z]{1}',
        'label': 'TAN',
        'main_key': 'TAN',
    },
    'purchase_number': {                                   
        'regex': '\d{1,}',
        'label': 'Purchase_Order_Number',
        'main_key': 'Purchase_Order_Number',
    },
    'gstin': {
        'regex': '[A-Z0-9]{15}',
        'label': 'GSTIN',
        'main_key': 'GSTIN',
    },
    'tan': {
        'regex': '[A-Z]{4}[0-9][4]{A-Z}{1}',
        'label': 'TAN',
        'main_key': 'TAN',
    },
    'invoice_no': {
        'regex': '\d+|[A-Z]+\d+[A-Z]\s?(\d{2})?\s?\-\s?\d+|[A-Z]\s?\-?\s?\d+',
        'label': 'INVOICE_NO',
        'main_key': 'INVOICE_NO',
    },
    # 12345 | SA18Y 09 - 00205 | S - 168
    'invoice_date': {
        'regex': date_regex,
        'label': 'INVOICE_NO',
        'main_key': 'Invoice_Date',
    },
    'invoice_date_1': {
        'regex': date_regex,
        'label': 'Invoice_Date',
        'main_key': 'Invoice_Date',
    },
    # (MM.DD.YYYY) | MM - DDD - YYYY | MM -/ DD -/ YYYY
    'lrno': {
        'regex': '\d{1,2}\s?[(th)\-\.]?\s?([A-Za-z]{3,})\s?[\-\.]?\s?\d{4}|\d{1,2}\s?\.\s?\d{1,2}\s?\.\s?\d{2}|BY MONTH\s?\/?\s?\d{4}\s?\-\s?\d{4}',
        'label': 'L.R._No',
        'main_key': 'L.R._No',
    },
    'lrdt': {
        'regex': date_regex,
        'label': 'L.R._Dt',
        'main_key': 'L.R._Dt',
    },
    
}

number_regex = '(\,? ?\d+ ?)+(\. ?\d+)?(?!\d{0,} ?%)'

patterns_reversed = {
    'sgst': {
        'regex': number_regex,
        'label': 'SGST',
        'main_key': 'SGST_Amount',
    },
    'cgst': {
        'regex': number_regex,
        'label': 'CGST',
        'main_key': 'CGST_Amount',
    },
    'igst': {
        'regex': number_regex,
        #1 93 , 626 . 00
        'label': 'IGST',
        'main_key': 'IGST_Amount',
    },
    'sgst_percentage': {
        'regex': percentage_regex,
        'label': 'SGST',
        'main_key': 'SGSTPercentage',
    },
    'cgst_percentage': {
        'regex': percentage_regex,
        'label': 'CST',
        'main_key': 'CSTPercentage',
    },
    'igst_percentage': {
        'regex': percentage_regex,
        'label': 'IST',
        'main_key': 'ISTPercentage',
    },
    'total': {
        'regex': number_regex,
        'label': 'Gross_Total',
        'main_key': 'Gross_Total',
    },
}

In [55]:
result_keys = [
    'Tax_Invoice',
    'INVOICE_NO',
    'Ship_To',
    'Bill_To',
    'EssarGSTIN',
    'Hazira',
    'Invoice_Date',
    'PAN_NO',
    'TAN',
    'Purchase_Order_Number',
    'GSTIN',
    'L.R._No',
    'L.R._Dt',
    'Transport',
    'SERVICES_TAX_REG_NO',
    'SGST_Amount',
    'CGST_Amount',
    'IGST_Amount',
    'CGSTPercentage',
    'SGSTPercentage',
    'IGSTPercentage',
    'Gross_Total',
]

number_keys = [
    'SGST_Amount',
    'CGST_Amount',
    'IGST_Amount',
    'Gross_Total',
]

result = { 'items': [] }
for key in result_keys:
    result[SELECTED_LABEL + key] = None

for line, phrases in ssum.items():
    for index, phrase in enumerate(phrases):
        lookup = sres[line][index]
        if not isinstance(lookup, list):
            continue
            
        result_label = [ strip(label.split('#')[-1]) for label in result.keys() if result[label] is not None ]    
        labels = [ strip(word.split('#')[-1]) for word in lookup ]
        
        if 'taxinvoice' in labels and 'taxinvoice' not in result_label:
            taxinvoice_index = labels.index('taxinvoice')
            result[lookup[taxinvoice_index]] = True
 
        for pattern in patterns.values():
            # ignore already exist key in result
            if strip(pattern['main_key']) in result_label:
                continue

            if strip(pattern['label']) in labels:
                print(line, pattern['main_key'])
                # concatenate the remaining text
                label_index = labels.index(strip(pattern['label']))
                paragraph = concat(ssum, line, index)
                search_p = re.search(pattern['regex'], paragraph)
                if search_p is not None:
                    result[SELECTED_LABEL + pattern['main_key']] = search_p.group(0)
                    
for line in lines[::-1]:
    phrases = ssum[line]
    for index in range(len(phrases) - 1, -1, -1):
        phrase = phrases[index]
        lookup = sres[line][index]
        if not isinstance(lookup, list):
            continue
            
        result_label = [ strip(label.split('#')[-1]) for label in result.keys() if result[label] is not None ]    
        labels = [ strip(word.split('#')[-1]) for word in lookup ]
 
        for pattern in patterns_reversed.values():
            # ignore already exist key in result
            if strip(pattern['main_key']) in result_label:
                continue

            if strip(pattern['label']) in labels:
                print(line, pattern['main_key'])
                # concatenate the remaining text
                label_index = labels.index(strip(pattern['label']))
                paragraph = concat(ssum, line, index)
                search_p = re.search(pattern['regex'], paragraph)
                if search_p is not None:
                    key = SELECTED_LABEL + pattern['main_key']
                    result[key] = search_p.group(0)
                    if pattern['main_key'] in number_keys:
                        result[key] = to_float(result[key])


print(json.dumps(result, indent=2))

448 GSTIN
520 PAN_NO
823 INVOICE_NO
823 Invoice_Date
1769 Gross_Total
1726 SGST_Amount
1726 SGSTPercentage
1658 CGST_Amount
{
  "items": [],
  "http://bizlem.io/PurchaseOrderProcessing#Tax_Invoice": true,
  "http://bizlem.io/PurchaseOrderProcessing#INVOICE_NO": "1819960",
  "http://bizlem.io/PurchaseOrderProcessing#Ship_To": null,
  "http://bizlem.io/PurchaseOrderProcessing#Bill_To": null,
  "http://bizlem.io/PurchaseOrderProcessing#EssarGSTIN": null,
  "http://bizlem.io/PurchaseOrderProcessing#Hazira": null,
  "http://bizlem.io/PurchaseOrderProcessing#Invoice_Date": "25 / 12 / 2018",
  "http://bizlem.io/PurchaseOrderProcessing#PAN_NO": "ABIFS0267Q",
  "http://bizlem.io/PurchaseOrderProcessing#TAN": null,
  "http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number": null,
  "http://bizlem.io/PurchaseOrderProcessing#GSTIN": "24ABIFS0267Q1ZO",
  "http://bizlem.io/PurchaseOrderProcessing#L.R._No": null,
  "http://bizlem.io/PurchaseOrderProcessing#L.R._Dt": null,
  "http://bizlem.io/

In [67]:
a = {'a': 1, 'b': 2, 'c': 3}
b = [1, 2, 3]
print(b[::-1])

[3, 2, 1]


In [380]:
header_ids = ['srno', 'descriptionofservices', 'unit', 'quantity', 'rate', 'amount', 'hsncode']
selected = []
header_map = {}

def overlap(x1, x2):
    left_x1, right_x1 = x1
    left_x2, right_x2 = x2
    return (min(right_x1, right_x2) - max(left_x1, left_x2)) / min(right_x1 - left_x1, right_x2 - left_x2)

for line, phrases in ssum.items():
    for index, phrase in enumerate(phrases):
        lookup = sres[line][index]
        if not isinstance(lookup, list):
            continue
            
        labels = [ strip(word.split('#')[-1]) for word in lookup ]
        
        for header_id in header_ids:
            if header_id in selected:
                continue
                
            if header_id in labels:
                label_index = labels.index(header_id)
                word_arr = slist[line][index]
                left_x, right_x = get_x_range(word_arr)
                
                same = False
                for _, hleft_x, hright_x in header_map.values():
                    if overlap((hleft_x, hright_x), (left_x, right_x)) >= 0.5:
                        same = True
                        break
                        
                if not same:
                    header_map[lookup[label_index]] = (line, left_x, right_x)
                    selected.append(header_id)
                    
print(header_map)

{'http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES': (312, 414, 476), 'http://bizlem.io/PurchaseOrderProcessing#UNIT': (341, 515, 559), 'http://bizlem.io/PurchaseOrderProcessing#AMOUNT': (619, 62, 148)}


In [381]:
header_pos = []
start_line = 0

for header, (line, left_x, right_x) in header_map.items():
    header_pos.append((header, line, [left_x, right_x]))
    if line > start_line:
        start_line = line
    
header_pos.sort(key=lambda x: x[-1][0])
header_temp = [ (pos[-1][0], pos[-1][1]) for pos in header_pos ]

    
for i in range(len(header_pos)):
    if i > 0:
        header_pos[i][2][0] = header_temp[i - 1][1]
    else:
        header_pos[i][2][0] = 0
        
    if i < len(header_pos) - 1:
        header_pos[i][2][1] = header_temp[i + 1][0]
    else:
        header_pos[i][2][1] = 1000000
        

In [382]:
slist_flat = {}
for line, words_list in slist.items():
    slist_flat[line] = []
    for words in words_list:
        for word in words:
            slist_flat[line].append(word)

In [370]:
print(header_pos)

[('http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES', 501, [0, 457]), ('http://bizlem.io/Quantity#Quantity', 490, [269, 531]), ('http://bizlem.io/PurchaseOrderProcessing#RATE', 501, [497, 678]), ('http://bizlem.io/PurchaseOrderProcessing#AMOUNT', 501, [554, 1000000])]


In [383]:
items = []
item = {}

header_check = {}
for header, _, _ in header_pos:
    header_check[header] = None

found = False
for line in lines:
    if line <= start_line:
        continue
    words = slist_flat[line]
    
    found = False
    for word_index in words:
        left_x, _, right_x, _ = boxes[word_index]
        
        found = False
        for header, header_line, pos in header_pos:
            if left_x >= pos[0] and right_x <= pos[1]:
                desc = data[word_index]['description']
                found = True
                
                #print(line, desc, left_x, right_x, header)
                if header not in item:
                    item[header] = desc
                    
                if 'descriptionofservices' not in strip(header):
                    if header_check[header] is None:
                        header_check[header] = line
                    elif header_check[header] != line:
                        items.append(item)
                        item = {}
                        for h in header_check.keys():
                            header_check[h] = None
                    else:
                        item[header] += ' ' + desc
                else:
                    item[header] += ' ' + desc
                break
        if not found:
            break
            
    if not found:
        break

if len(item.keys()) > 0:
    items.append(item)
    
print(json.dumps(items, indent=2))

[
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "0150103"
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "N2000HZ1 18 - 19"
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "CODE",
    "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "NE NE",
    "http://bizlem.io/PurchaseOrderProcessing#UNIT": "FOR BARON INFRACON LIMITED"
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "ELIGIBLE"
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "NOT",
    "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "ELIGIBLE ELIGIBLE C",
    "http://bizlem.io/PurchaseOrderProcessing#UNIT": "Director / Authorise Signatory"
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "PAYABLE",
    "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "EXEMPTED EXEMPTED"
  },
  {
    "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "that Particulars given above are correct Sdl . He GS

In [235]:
result['items'] = items

print(json.dumps(result, indent=2))

{
  "items": [
    {
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "Carpenta Carpenta Services",
      "http://bizlem.io/PurchaseOrderProcessing#RATE": ". 00"
    },
    {
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "Related Related to furnituel Lola come to col cleb as po annexhle A ) ( Pope false Ceiliop 1000463217 Wiesh TOTAL IGST",
      "http://bizlem.io/PurchaseOrderProcessing#RATE": "N2000HZ117 - 18 - 0119893"
    },
    {
      "http://bizlem.io/PurchaseOrderProcessing#RATE": "% 69992 . eo",
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "TOTAL TOTAL TAX GRAND TOTAL"
    },
    {
      "http://bizlem.io/PurchaseOrderProcessing#RATE": ", 839 . 00"
    },
    {
      "http://bizlem.io/PurchaseOrderProcessing#RATE": "CHAIVU NDA INTERIOR",
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "Certifies Certifies that partitulars given above are correct"
    },
    {
      "http

In [57]:
def process_json(path):
    with open(path) as f:
        data = json.load(f)
        data_sum = data[0]
        data = data[1:]

        max_x = 0
        max_y = 0
        
    # calculate boxes
    boxes = []
    for obj in data:
        boxes.append(get_box(obj))

    # find max_x and max_y
    for i, obj in enumerate(data):
        _, _, right_x, bot_y = boxes[i]
        max_x = max(max_x, right_x)
        max_y = max(max_y, bot_y)
        
    print(max_x, max_y, len(data))
    height = max_y + 1
    width = max_x + 1
    coor = [[ None for x in range(width) ] for y in range(height)] 

    # map to coor
    for index, obj in enumerate(data):
        left_x, top_y, right_x, bot_y = boxes[index]

        for y in range(top_y, bot_y + 1):
            for x in range(left_x, right_x + 1):
                if coor[y][x] is not None:
                    continue
                coor[y][x] = index

    slist = {}
    visited = []

    MARGIN_X = 5
    MARGIN_Y = 8

    print('processing matrix...')
    for x in range(width):
        for y in range(height - 1, -1, -1):
            if coor[y][x] is None or coor[y][x] in visited:
                continue
                
            index_obj = coor[y][x]
            left_x, top_y, _, bot_y = boxes[index_obj]
            obj = data[index_obj]
            visited.append(index_obj)
            
            # test range y within MARGIN_Y
            row = y
            MARGIN_Y = int((bot_y - top_y) * 0.6)
            for i in range(y - MARGIN_Y, y + MARGIN_Y + 1):
                if i in slist.keys():
                    row = i
                    break
            
            if row not in slist.keys():
                slist[row] = []
                
            if len(slist[row]) > 0:
                prev_words = slist[row][-1]
                prev_word = prev_words[-1]
                pw_left_x, pw_top_y, pw_right_x, pw_bot_y = boxes[prev_word]
                
                MARGIN_X = int((pw_bot_y - pw_top_y) * 0.7)
                
                if pw_right_x + MARGIN_X >= left_x:
                    prev_words.append(index_obj)
                    continue
                
            slist[row].append([ index_obj ])
    print('done processing matrix')

    lines = list(slist.keys())
    lines.sort()

    ssum = {}
    sres = {}

    for line in lines:
        words_list = slist[line]
        phrases = []
        for words in words_list:
            phrase = [ data[index]['description'] for index in words ]
            phrases.append(' '.join(phrase))
        ssum[line] = phrases
        sres[line] = phrases.copy() 
    def request_word(word):
        res = []
        try:
            r = requests.post(URL, data=word.encode('utf-8'), headers={'Content-Type': 'application/pdf'})
            r = r.json()
            
            for obj in r:
                if LABEL in obj:
                    res.append(obj[LABEL][0]['@id'])
            return res
        except Exception as e:
            print(word)
            print(e)
            return res
        
    def request_label(ssum, line, index, sres):
        res = request_word(ssum[line][index])
        sres[line][index] = res
        
    def request_line(ssum, line, sres):
        threads = [ threading.Thread(target=request_label, args=(ssum, line, index, sres)) for index in range(len(ssum[line])) ]
        for thread in threads:
            thread.start()
            
        for thread in threads:
            thread.join()
            
    def request_all(ssum, sres):
        threads = [ threading.Thread(target=request_line, args=(ssum, line, sres)) for line in ssum.keys() ]
        for thread in threads:
            thread.start()
            
        for thread in threads:
            thread.join()
        
    print('requesting...')
    request_all(ssum, sres)
    print('finish requesting...')


    def concat(ssum, line, index):                            
        paragraph = ''                                         
        for l, phrases in ssum.items():                         
            if l < line:                                       
                continue
            if l == line:
                paragraph += ' '.join(phrases[index:])
            else:
                sentence = ' '.join(phrases)
                paragraph += '\n' + sentence
        return paragraph

    def strip(s):
        return ''.join(re.split('[^a-zA-Z0-9]', s.lower()))

    # processing text
    date_regex = '\d{1,2}\s?[\.\-\/\:]\s?\d{1,2}\s?[\.\-\/\:]\s?\d{2,4}|\d{1,2}\s?\-?\s?[A-Za-z]{3,}\s?\-?\s?(\d{2}|\d{4})'
    number_regex = '(\,? ?\d+ ?)+(\. ?\d+)?(?!\d{0,} ?%)'
    percentage_regex = '\d+\s?%|\d+\s?\.\s?%'

    # processing text
    patterns = {                                               
        'pan': {                                               
            'regex': '[A-Z]{5}[0-9]{4}[A-Z]{1}',               
            'label': 'PAN_NO',
            'main_key': 'PAN_NO',
        },
        'tan': {                                               
            'regex': '[A-Z]{4}[0-9]{5}[A-Z]{1}',
            'label': 'TAN',
            'main_key': 'TAN',
        },
        'purchase_number': {                                   
            'regex': '\d{1,}',
            'label': 'Purchase_Order_Number',
            'main_key': 'Purchase_Order_Number',
        },
        'gstin': {
            'regex': '[A-Z0-9]{15}',
            'label': 'GSTIN',
            'main_key': 'GSTIN',
        },
        'tan': {
            'regex': '[A-Z]{4}[0-9][4]{A-Z}{1}',
            'label': 'TAN',
            'main_key': 'TAN',
        },
        'invoice_no': {
            'regex': '\d+|[A-Z]+\d+[A-Z]\s?(\d{2})?\s?\-\s?\d+|[A-Z]\s?\-?\s?\d+',
            'label': 'INVOICE_NO',
            'main_key': 'INVOICE_NO',
        },
        # 12345 | SA18Y 09 - 00205 | S - 168
        'invoice_date': {
            'regex': date_regex,
            'label': 'INVOICE_NO',
            'main_key': 'Invoice_Date',
        },
        'invoice_date_1': {
            'regex': date_regex,
            'label': 'Invoice_Date',
            'main_key': 'Invoice_Date',
        },
        # (MM.DD.YYYY) | MM - DDD - YYYY | MM -/ DD -/ YYYY
        'lrno': {
            'regex': '\d{1,2}\s?[(th)\-\.]?\s?([A-Za-z]{3,})\s?[\-\.]?\s?\d{4}|\d{1,2}\s?\.\s?\d{1,2}\s?\.\s?\d{2}|BY MONTH\s?\/?\s?\d{4}\s?\-\s?\d{4}',
            'label': 'L.R._No',
            'main_key': 'L.R._No',
        },
        'lrdt': {
            'regex': date_regex,
            'label': 'L.R._Dt',
            'main_key': 'L.R._Dt',
        },

    }

    number_regex = '(\,? ?\d+ ?)+(\. ?\d+)?(?!\d{0,} ?%)'

    patterns_reversed = {
        'sgst': {
            'regex': number_regex,
            'label': 'SGST',
            'main_key': 'SGST_Amount',
        },
        'cgst': {
            'regex': number_regex,
            'label': 'CGST',
            'main_key': 'CGST_Amount',
        },
        'igst': {
            'regex': number_regex,
            #1 93 , 626 . 00
            'label': 'IGST',
            'main_key': 'IGST_Amount',
        },
        'sgst_percentage': {
            'regex': percentage_regex,
            'label': 'SGST',
            'main_key': 'SGSTPercentage',
        },
        'cgst_percentage': {
            'regex': percentage_regex,
            'label': 'CST',
            'main_key': 'CSTPercentage',
        },
        'igst_percentage': {
            'regex': percentage_regex,
            'label': 'IST',
            'main_key': 'ISTPercentage',
        },
        'total': {
            'regex': number_regex,
            'label': 'Gross_Total',
            'main_key': 'Gross_Total',
        },
    }
    result_keys = [
        'Tax_Invoice',
        'INVOICE_NO',
        'Ship_To',
        'Bill_To',
        'EssarGSTIN',
        'Hazira',
        'Invoice_Date',
        'PAN_NO',
        'TAN',
        'Purchase_Order_Number',
        'GSTIN',
        'L.R._No',
        'L.R._Dt',
        'Transport',
        'SERVICES_TAX_REG_NO',
        'SGST_Amount',
        'CGST_Amount',
        'IGST_Amount',
        'CGSTPercentage',
        'SGSTPercentage',
        'IGSTPercentage',
        'Gross_Total',
    ]

    number_keys = [
        'SGST_Amount',
        'CGST_Amount',
        'IGST_Amount',
        'Gross_Total',
    ]

    result = { 'items': [] }
    for key in result_keys:
        result[SELECTED_LABEL + key] = None

    for line, phrases in ssum.items():
        for index, phrase in enumerate(phrases):
            lookup = sres[line][index]
            if not isinstance(lookup, list):
                continue

            result_label = [ strip(label.split('#')[-1]) for label in result.keys() if result[label] is not None ]    
            labels = [ strip(word.split('#')[-1]) for word in lookup ]

            if 'taxinvoice' in labels and 'taxinvoice' not in result_label:
                taxinvoice_index = labels.index('taxinvoice')
                result[lookup[taxinvoice_index]] = True

            for pattern in patterns.values():
                # ignore already exist key in result
                if strip(pattern['main_key']) in result_label:
                    continue

                if strip(pattern['label']) in labels:
                    print(line, pattern['main_key'])
                    # concatenate the remaining text
                    label_index = labels.index(strip(pattern['label']))
                    paragraph = concat(ssum, line, index)
                    search_p = re.search(pattern['regex'], paragraph)
                    if search_p is not None:
                        result[SELECTED_LABEL + pattern['main_key']] = search_p.group(0)

    for line in lines[::-1]:
        phrases = ssum[line]
        for index in range(len(phrases) - 1, -1, -1):
            phrase = phrases[index]
            lookup = sres[line][index]
            if not isinstance(lookup, list):
                continue

            result_label = [ strip(label.split('#')[-1]) for label in result.keys() if result[label] is not None ]    
            labels = [ strip(word.split('#')[-1]) for word in lookup ]

            for pattern in patterns_reversed.values():
                # ignore already exist key in result
                if strip(pattern['main_key']) in result_label:
                    continue

                if strip(pattern['label']) in labels:
                    print(line, pattern['main_key'])
                    # concatenate the remaining text
                    label_index = labels.index(strip(pattern['label']))
                    paragraph = concat(ssum, line, index)
                    search_p = re.search(pattern['regex'], paragraph)
                    if search_p is not None:
                        key = SELECTED_LABEL + pattern['main_key']
                        result[key] = search_p.group(0)
                        if pattern['main_key'] in number_keys:
                            result[key] = to_float(result[key])

    header_ids = ['srno', 'descriptionofservices', 'unit', 'quantity', 'rate', 'amount', 'hsncode']
    selected = []
    header_map = {}

    def overlap(x1, x2):
        left_x1, right_x1 = x1
        left_x2, right_x2 = x2
        return (min(right_x1, right_x2) - max(left_x1, left_x2)) / min(right_x1 - left_x1, right_x2 - left_x2)

    for line, phrases in ssum.items():
        for index, phrase in enumerate(phrases):
            lookup = sres[line][index]
            if not isinstance(lookup, list):
                continue
                
            labels = [ strip(word.split('#')[-1]) for word in lookup ]
            
            for header_id in header_ids:
                if header_id in selected:
                    continue
                    
                if header_id in labels:
                    label_index = labels.index(header_id)
                    word_arr = slist[line][index]
                    left_x, right_x = get_x_range(word_arr, boxes)
                    
                    same = False
                    for _, hleft_x, hright_x in header_map.values():
                        if overlap((hleft_x, hright_x), (left_x, right_x)) >= 0.5:
                            same = True
                            break
                            
                    if not same:
                        header_map[lookup[label_index]] = (line, left_x, right_x)
                        selected.append(header_id)

    header_pos = []
    start_line = 0

    for header, (line, left_x, right_x) in header_map.items():
        header_pos.append((header, line, [left_x, right_x]))
        if line > start_line:
            start_line = line
        
    header_pos.sort(key=lambda x: x[-1][0])
    header_temp = [ (pos[-1][0], pos[-1][1]) for pos in header_pos ]

        
    for i in range(len(header_pos)):
        if i > 0:
            header_pos[i][2][0] = header_temp[i - 1][1]
        else:
            header_pos[i][2][0] = 0
            
        if i < len(header_pos) - 1:
            header_pos[i][2][1] = header_temp[i + 1][0]
        else:
            header_pos[i][2][1] = 1000000

    slist_flat = {}
    for line, words_list in slist.items():
        slist_flat[line] = []
        for words in words_list:
            for word in words:
                slist_flat[line].append(word)

    items = []
    item = {}

    header_check = {}
    for header, _, _ in header_pos:
        header_check[header] = None

    found = False
    for line in lines:
        if line <= start_line:
            continue
        words = slist_flat[line]
        
        found = False
        for word_index in words:
            left_x, _, right_x, _ = boxes[word_index]
            
            found = False
            for header, header_line, pos in header_pos:
                if left_x >= pos[0] and right_x <= pos[1]:
                    desc = data[word_index]['description']
                    found = True
                    
                    #print(line, desc, left_x, right_x, header)
                    if header not in item:
                        item[header] = desc
                        
                    if 'descriptionofservices' not in strip(header):
                        if header_check[header] is None:
                            header_check[header] = line
                        elif header_check[header] != line:
                            items.append(item)
                            item = {}
                            for h in header_check.keys():
                                header_check[h] = None
                        else:
                            item[header] += ' ' + desc
                    else:
                        item[header] += ' ' + desc
                    break
            if not found:
                break
                
        if not found:
            break

    if len(item.keys()) > 0:
        items.append(item)
    
    result['items'] = items

    return result

In [58]:
r = process_json('p4materials/c7.json')

print(json.dumps(r, indent=2))

770 1092 409
processing matrix...
done processing matrix
requesting...
finish requesting...
345 GSTIN
358 PAN_NO
389 Invoice_Date
389 INVOICE_NO
675 Gross_Total
395 CGST_Amount
{
  "items": [
    {
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "DEPLOYMENT DEPLOYMENT OF ) 1 . EXCAVATOR ( TATA HITACHI 9973 18",
      "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "27 , 225 3 , 56 , 950",
      "http://bizlem.io/PurchaseOrderProcessing#UNIT": "1 100"
    },
    {
      "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": ", 02 , 500 9 % 27 , 225 9 %",
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "WORKING WORKING DAYS 01 - 11 - 2018 TO 28 - 11 - 2018",
      "http://bizlem.io/PurchaseOrderProcessing#UNIT": "TOTAL"
    },
    {
      "http://bizlem.io/PurchaseOrderProcessing#AMOUNT": "SIT",
      "http://bizlem.io/PurchaseOrderProcessing#DESCRIPTION_OF_SERVICES": "N208 N208 @ HZ118 - - 19 0150334 Total Amount",
      "http://bizlem.

In [61]:
#         if 'cgst' in labels and 'cgst' not in result_label:
#             cgst_index = labels.index('cgst')
#             result[lookup[cgst_index]] = ' '.join(phrases[index:])
        
#         if 'sgst' in labels and 'sgst' not in result_label:
#             sgst_index = labels.index('sgst')
#             result[lookup[sgst_index]] = ' '.join(phrases[index:])
            
#         if 'invoiceno' in labels and 'invoiceno' not in result_label:
#             invoiceno_index = labels.index('invoiceno')
#             i = index + 1
#             r = phrases[index]
#             while i < len(phrases) and len(sres[line][i]) == 0:
#                 r += ' ' + phrases[i]
#                 i += 1
                
#             if i == index + 1:
#                 next_line = lines[lines.index(line) + 1]
#                 current_row = slist[line]
#                 current_left_x, current_right_x = get_x_range(current_row[index])
                
#                 next_row = slist[next_line]
#                 for next_row_i, word_list in enumerate(next_row):
#                     next_left_x, next_right_x = get_x_range(word_list)
#                     if next_left_x >= current_left_x - 5 and next_left_x <= current_left_x + 5:
#                         r += ' ' + ssum[next_line][next_row_i]
                        
#             result[lookup[invoiceno_index]] = r
                
#         if 'invoicedate' in labels and 'invoicedate' not in result_label:
#             invoicedate_index = labels.index('invoicedate')
#             i = index + 1
#             r = phrases[index]
#             while i < len(phrases) and len(sres[line][i]) == 0:
#                 r += ' ' + phrases[i]
#                 i += 1
                
#             if i == index + 1:
#                 next_line = lines[lines.index(line) + 1]
#                 current_row = slist[line]
#                 current_left_x, current_right_x = get_x_range(current_row[index])
                
#                 next_row = slist[next_line]
#                 for next_row_i, word_list in enumerate(next_row):
#                     next_left_x, next_right_x = get_x_range(word_list)
#                     if next_left_x >= current_left_x - 5 and next_left_x <= current_left_x + 5:
#                         r += ' ' + ssum[next_line][next_row_i]
                        
#             result[lookup[invoicedate_index]] = r